  import paho.mqtt.client as mqtt
import ssl
host = "localhost"
port = 1883
topic = "tagsLive" 
def on_connect(client, userdata, flags, rc):
    print(mqtt.connack_string(rc))
# callback triggered by a new Pozyx data packet
def on_message(client, userdata, msg):
    print("Positioning update:", msg.payload.decode())
def on_subscribe(client, userdata, mid, granted_qos):
    print("Subscribed to topic!")
client = mqtt.Client()
# set callbacks
client.on_connect = on_connect
client.on_message = on_message
client.on_subscribe = on_subscribe
client.connect(host, port=port)
client.subscribe(topic)
# works blocking, other, non-blocking, clients are available too.
#client.loop_forever()

In [4]:
from typing import Callable

import paho.mqtt.client as mqtt
import ssl
import json

from time import time

import tkinter as tk
from tkinter import Tk, BOTH
from tkinter.ttk import Frame, Button, Style, Entry


start_time = time()

def on_connect(client, userdata, flags, rc):
    print(mqtt.connack_string(rc))


def on_subscribe(client, userdata, mid, granted_qos):
    print("Subscribed to topic!")


def on_disconnect(*args, **kwargs):
    print("Disconnected from MQTT")
    # print(mqtt.connack_string(rc))


def on_unsubscribe(*args, **kwargs):
    print("Unsubscribed to topic!")


def connect_locally(on_message_handler: Callable, on_connect_handler: Callable=None,
                    on_subscribe_handler: Callable=None, on_unsubscribe_handler: Callable=None,
                    on_disconnect_handler: Callable=None) -> mqtt.Client:
    host = "localhost"
    port = 1883
    topic = "tagsLive"

    mqtt_client = mqtt.Client()

    mqtt_client.on_message = on_message_handler
    mqtt_client.on_connect = on_connect_handler if on_connect_handler is not None else on_connect
    mqtt_client.on_subscribe = on_subscribe_handler if on_subscribe_handler is not None else on_subscribe
    mqtt_client.on_disconnect = on_disconnect_handler if on_disconnect_handler is not None else on_disconnect
    mqtt_client.on_unsubscribe = on_unsubscribe_handler if on_unsubscribe_handler is not None else on_unsubscribe

    mqtt_client.connect(host, port=port)
    mqtt_client.subscribe(topic)

    return mqtt_client


def connect_cloud(on_message_handler: Callable, on_connect_handler: Callable=None,
                  on_subscribe_handler: Callable=None)-> mqtt.Client:
    host = "mqtt.cloud.pozyxlabs.com"
    port = 443
    topic = "5a8efa70831eaa000572e593"
    username = "5a8efa70831eaa000572e593"
    password = "14e20680-7324-4f83-ae13-05a740a7f803"

    mqtt_client = mqtt.Client(transport="websockets")

    mqtt_client.username_pw_set(username, password=password)

    mqtt_client.tls_set_context(context=ssl.create_default_context())

    mqtt_client.on_message = on_message_handler
    mqtt_client.on_connect = on_connect_handler if on_connect_handler is not None else on_connect
    mqtt_client.on_subscribe = on_subscribe_handler if on_subscribe_handler is not None else on_subscribe

    mqtt_client.connect(host, port=port)
    mqtt_client.subscribe(topic)

    return mqtt_client


class NoRepetitionPrinter:
    def __init__(self):
        self._previous_message = None
        self.periodic_prints = {}

    def print(self, *args):
        if " ".join(args) == self._previous_message:
            pass
        else:
            self._previous_message = " ".join(args)
            print(*args)

    def print_periodic(self, *args):
        string = " ".join(args)
        now = time()
        if string in self.periodic_prints:
            if now - self.periodic_prints[string] > 10:
                print("{:4f}: {}".format(now - start_time, string))
                self.periodic_prints[string] = now
        else:
            print("{:4f}: {}".format(now - start_time, string))
            self.periodic_prints[string] = now


no_repetition_printer = NoRepetitionPrinter()


class Example(Frame):
    def __init__(self, root, use_cloud=False, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._tk_root = root

        self._use_cloud = use_cloud
        self.initUI()

        self.file_pointer = None
        self.mqtt_client = None

        self.update_mqtt()

    def reconnect_mqtt(self):
        # try:
        try:
            if self._use_cloud:
                self.mqtt_client = connect_cloud(self.on_message_cloud, on_disconnect_handler=self.on_mqtt_disconnected)
                no_repetition_printer.print("Made connection to cloud MQTT, will connect on first readout")
            else:
                self.mqtt_client = connect_locally(self.on_message_local, on_disconnect_handler=self.on_mqtt_disconnected)
                no_repetition_printer.print("Made connection to local MQTT, will connect on first readout")
            # self.mqtt_client.loop(timeout=0.0)
        except ConnectionRefusedError:
            no_repetition_printer.print("No MQTT broker currently available")

    def update_mqtt(self):
        if self.mqtt_client is None:
            self.reconnect_mqtt()
            self.after(100, self.update_mqtt)
        else:
            no_repetition_printer.print_periodic("MQTT still connected")
            self.mqtt_client.loop()
            self.after(0, self.update_mqtt)

    def on_mqtt_disconnected(self, *args, **kwargs):
        no_repetition_printer.print("Disconnected from MQTT")
        self.mqtt_client = None

    def initUI(self):
        self.style = Style()
        self.style.theme_use("default")

        self.logging = False

        self.master.title("Logging")
        self.pack(fill=BOTH, expand=1)

        self.entry = Entry(self)

        self.entry.place(x=50, y=20)

        self.entry.insert(0, "log_name_test.log")

        self.button_text = tk.StringVar()
        self.button_text.set("Start logging")

        self.button = Button(self, textvariable=self.button_text,
                            command=self.start_logging)
        self.button.place(x=50, y=50)
        
    def update(self):
        super(Example, self).update()
        print("Hehehe")

    def start_logging(self):
        print("Started logging to {}".format(self.entry.get()))
        self.logging = True
        self.button_text.set("Stop logging")
        self.button["command"] = self.stop_logging
        self.file_pointer = open(self.entry.get(), 'a')
        self.after(10, self.update)

    def stop_logging(self):
        print("Finished logging")
        self.logging = False
        self.button_text.set("Start logging")
        self.button["command"] = self.start_logging

        if self.file_pointer is not None:
            self.file_pointer.close()
            self.file_pointer = None

    # callback triggered by a new Pozyx data packet
    def on_message_local(self, client, userdata, msg):
        no_repetition_printer.print_periodic("Receiving MQTT messages")
        if not self.logging or self.file_pointer is None:
            return
        try:
            tag_data_json = msg.payload.decode()
            tag_data = json.loads(tag_data_json)
            print(tag_data)
            tag_time = float(tag_data["timestamp"])
            if time() - tag_time <= 0.5:
                self.file_pointer.write(tag_data_json + "\n")
        except Exception as e:
            print("No valid tag data: {}".format(e))

    # callback triggered by a new Pozyx data packet
    def on_message_cloud(self, client, userdata, msg):
        if not self.logging or self.file_pointer is None:
            return
        try:
            tags_data_json = msg.payload.decode()
            tags_data = json.loads(tags_data_json)
            for tag_data in tags_data:
                tag_time = float(tag_data["timestamp"])
                if time() - tag_time <= 1:
                    self.file_pointer.write(json.dumps(tag_data) + "\n")

        except Exception as e:
            print("No valid tag data: {}".format(e))



def main():
    root = Tk()
    root.geometry("250x150+300+300")
    app = Example(root, use_cloud=False)
    root.mainloop()


if __name__ == '__main__':
    main()

Made connection to local MQTT, will connect on first readout
0.225644: MQTT still connected
Connection Accepted.
Subscribed to topic!
0.386325: Receiving MQTT messages
10.252174: MQTT still connected
10.706531: Receiving MQTT messages
20.437257: MQTT still connected
20.796760: Receiving MQTT messages
30.707148: MQTT still connected
30.818848: Receiving MQTT messages
40.839832: Receiving MQTT messages
40.840934: MQTT still connected
50.901689: Receiving MQTT messages
50.907672: MQTT still connected
60.938386: Receiving MQTT messages
60.938650: MQTT still connected
70.999031: Receiving MQTT messages
71.000031: MQTT still connected
81.229877: Receiving MQTT messages
81.231040: MQTT still connected
91.399769: Receiving MQTT messages
91.400766: MQTT still connected
101.499673: Receiving MQTT messages
101.500671: MQTT still connected
111.855921: Receiving MQTT messages
111.856110: MQTT still connected
122.161152: Receiving MQTT messages
122.162226: MQTT still connected
132.269755: Receiving 

{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181515.59, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': -3051, 'y': 5673, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 210.4, 'rates': {'update': 1.28, 'success': 1.28}}}}
{'version': '1', 'tagId': '26395', 'success': False, 'errorCode': '0xc', 'timestamp': 1559181516.118}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181516.151}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181516.475, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 3700, 'y': 5327, 'z': 0}, 'orientation': {'yaw': 3.961, 'roll': 0.048, 'pitch': 1.605}, 'metrics': {'latency': 78.5, 'rates': {'update': 1.25, 'success': 1.25}}}}
{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181516.562, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7731, 'y': 8198, 'z': 0}, 'orientatio

{'version': '1', 'tagId': '26395', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181523.505}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181523.868}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181524.213, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 3627, 'y': 5259, 'z': 0}, 'orientation': {'yaw': 3.966, 'roll': 0.021, 'pitch': 1.487}, 'metrics': {'latency': 79.8, 'rates': {'update': 1.18, 'success': 1.18}}}}
{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181524.306, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7702, 'y': 8153, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 133.2, 'rates': {'update': 1.18, 'success': 1.1}}}}
{'version': '1', 'tagId': '26395', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181524.444}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CO

{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181532.177, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7674, 'y': 8152, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 166, 'rates': {'update': 1.16, 'success': 1.13}}}}
{'version': '1', 'tagId': '26395', 'success': True, 'timestamp': 1559181532.351, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 4675, 'y': 5969, 'z': 0}, 'orientation': {'yaw': 4.253, 'roll': -0.268, 'pitch': 1.496}, 'metrics': {'latency': 74.4, 'rates': {'update': 1.17, 'success': 0.59}}}}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181532.432}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181532.822, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 3681, 'y': 5322, 'z': 0}, 'orientation': {'yaw': 4.057, 'roll': 0.023, 'pitch': 1.571}, 'metrics': {'latency': 79.8, 'rates': {'update': 1.1

{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181539.747, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7605, 'y': 8141, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 144.8, 'rates': {'update': 1.19, 'success': 1.17}}}}
{'version': '1', 'tagId': '26395', 'success': True, 'timestamp': 1559181539.927, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 4826, 'y': 5906, 'z': 0}, 'orientation': {'yaw': 4.174, 'roll': -0.239, 'pitch': 1.509}, 'metrics': {'latency': 78.4, 'rates': {'update': 1.19, 'success': 0.62}}}}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181540.014}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181540.373, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 3720, 'y': 5263, 'z': 0}, 'orientation': {'yaw': 4.068, 'roll': 0.028, 'pitch': 1.587}, 'metrics': {'latency': 79, 'rates': {'update': 1.2

...

{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181758.584}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181758.907, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 3719, 'y': 5293, 'z': 0}, 'orientation': {'yaw': 5.68, 'roll': 0.029, 'pitch': 1.644}, 'metrics': {'latency': 78.7, 'rates': {'update': 1.23, 'success': 1.23}}}}
648.085722: Receiving MQTT messages
{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181759.023, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7818, 'y': 8165, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 149.8, 'rates': {'update': 1.23, 'success': 1.2}}}}
648.087718: MQTT still connected
{'version': '1', 'tagId': '26395', 'success': False, 'errorCode': '0xc', 'timestamp': 1559181759.476}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181759.512}
{'versio

{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181765.855}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181766.244, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 3727, 'y': 5432, 'z': 0}, 'orientation': {'yaw': 5.786, 'roll': 0.015, 'pitch': 1.597}, 'metrics': {'latency': 77.4, 'rates': {'update': 1.24, 'success': 1.24}}}}
{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181766.327, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7705, 'y': 8141, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 156.6, 'rates': {'update': 1.25, 'success': 1.09}}}}
{'version': '1', 'tagId': '26395', 'success': True, 'timestamp': 1559181766.513, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 5441, 'y': 6329, 'z': 0}, 'orientation': {'yaw': 4.605, 'roll': -0.281, 'pitch': 1.692}, 'metrics': {'latency': 79.8, 'rates': {'update': 1

{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181771.638}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181772.001, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7759, 'y': 6327, 'z': 0}, 'orientation': {'yaw': 5.728, 'roll': -0.041, 'pitch': 1.568}, 'metrics': {'latency': 82.3, 'rates': {'update': 1.32, 'success': 1.32}}}}
{'version': '1', 'tagId': '26457', 'success': False, 'errorCode': '0x0', 'timestamp': 1559181772.348}
{'version': '1', 'tagId': '26395', 'success': True, 'timestamp': 1559181772.353, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7733, 'y': 7127, 'z': 0}, 'orientation': {'yaw': 5.502, 'roll': -0.293, 'pitch': 1.424}, 'metrics': {'latency': 83.8, 'rates': {'update': 1.31, 'success': 1.09}}}}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181772.465}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 15591

{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181777.936, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7690, 'y': 8201, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 165.7, 'rates': {'update': 1.36, 'success': 1.25}}}}
{'version': '1', 'tagId': '26395', 'success': True, 'timestamp': 1559181778.152, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 8884, 'y': 7551, 'z': 0}, 'orientation': {'yaw': 1.422, 'roll': -0.242, 'pitch': 1.489}, 'metrics': {'latency': 78.8, 'rates': {'update': 1.35, 'success': 1.24}}}}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181778.236}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181778.589, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 8270, 'y': 7274, 'z': 0}, 'orientation': {'yaw': 0.664, 'roll': -0.13, 'pitch': 1.635}, 'metrics': {'latency': 82.8, 'rates': {'update': 1

{'version': '1', 'tagId': '26457', 'success': True, 'timestamp': 1559181783.704, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 7652, 'y': 8157, 'z': 0}, 'orientation': {'yaw': 3.474, 'roll': 0, 'pitch': 0.182}, 'metrics': {'latency': 153.7, 'rates': {'update': 1.38, 'success': 1.33}}}}
{'version': '1', 'tagId': '26395', 'success': True, 'timestamp': 1559181783.867, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 8909, 'y': 7390, 'z': 0}, 'orientation': {'yaw': 1.005, 'roll': -0.166, 'pitch': 1.337}, 'metrics': {'latency': 83.8, 'rates': {'update': 1.38, 'success': 1.32}}}}
{'version': '1', 'tagId': '26382', 'success': False, 'errorCode': 'NO_CONN', 'timestamp': 1559181783.957}
{'version': '1', 'tagId': '26383', 'success': True, 'timestamp': 1559181784.344, 'data': {'tagData': {}, 'anchorData': [], 'coordinates': {'x': 8191, 'y': 7348, 'z': 0}, 'orientation': {'yaw': 0.857, 'roll': 0.057, 'pitch': 1.702}, 'metrics': {'latency': 77.1, 'rates': {'update': 1

In [3]:
import paho.mqtt.client as mqtt
import json
import datetime

from time import time

import tkinter as tk
from tkinter import Tk, BOTH
from tkinter.ttk import Frame, Button, Style, Entry


def on_connect(client, userdata, flags, rc):
    print(mqtt.connack_string(rc))

def on_subscribe(client, userdata, mid, granted_qos):
    print("Subscribed to topic!")


host = "localhost"
port = 1883
topic = "tagsLive"


class Example(Frame):

    def __init__(self):
        super().__init__()

        self.initUI()

        self.file_pointer = None

        self.mqtt_client = mqtt.Client()

        # set callbacks
        self.mqtt_client.on_connect = on_connect
        self.mqtt_client.on_message = self.on_message
        self.mqtt_client.on_subscribe = on_subscribe

        self.mqtt_client.connect(host, port=port)
        self.mqtt_client.subscribe(topic)

    def initUI(self):
        self.style = Style()
        self.style.theme_use("default")

        self.logging = False

        self.master.title("Logging")
        self.pack(fill=BOTH, expand=1)

        self.entry = Entry(self)

        self.entry.place(x=50, y=20)

        self.entry.insert(0, "log_name.log")

        self.button_text = tk.StringVar()
        self.button_text.set("Start logging")

        self.button = Button(self, textvariable=self.button_text,
                            command=self.start_logging)
        self.button.place(x=50, y=50)

    def start_logging(self):
        print("Started logging to {}".format(self.entry.get()))
        print(datetime.datetime.now())
        self.logging = True
        self.button_text.set("Stop logging")
        self.button["command"] = self.stop_logging
        self.file_pointer = open(self.entry.get(), 'a')
        self.after(1, self.update)

    def stop_logging(self):
        print("Finished logging")
        print(datetime.datetime.now())
        self.logging = False
        self.button_text.set("Start logging")
        self.button["command"] = self.start_logging

        if self.file_pointer is not None:
            self.file_pointer.close()
            self.file_pointer = None

    # callback triggered by a new Pozyx data packet
    def on_message(self, client, userdata, msg):
        if not self.logging or self.file_pointer is None:
            return
        try:
            tag_data_json = msg.payload.decode()
            tag_data = json.loads(tag_data_json)
            tag_time = float(tag_data["timestamp"])
            if time() - tag_time <= 0.5:
                self.file_pointer.write(tag_data_json + "\n")
        except Exception as e:
            print("No valid tag data: {}".format(e))

    def update(self):
        if self.logging:
            self.mqtt_client.loop()
            self.after(0, self.update)



def main():
    root = Tk()
    root.geometry("250x150+300+300")
    app = Example()
    root.mainloop()


if __name__ == '__main__':
    main()
    root = Tk()
    root.geometry("250x150+300+300")
    app = Example()
    root.mainloop()


if __name__ == '__main__':
    main()

Started logging to controltest3.json
2019-05-20 17:41:26.323101
Connection Accepted.
Subscribed to topic!
Finished logging
2019-05-20 17:42:04.852680
Started logging to controltest4.json
2019-05-20 17:42:40.407237
Connection Accepted.
Subscribed to topic!
Finished logging
2019-05-20 17:43:37.513469


In [1]:
import json
import pandas as pd

testing_json_log = pd.read_json('testjson.json', lines = True, orient = 'records')
#testing_json_log = pd.io.json.json_normalize(testing_json_log['data'], 'coordinates',

testing_json_log

id_timestamp_df = testing_json_log[['tagId','timestamp']]

In [2]:
new_json = testing_json_log['data'].to_json()
new_json_show = pd.read_json(new_json, orient = 'index')

coordinates_json = new_json_show['coordinates'].to_json()
coordinates_df = pd.read_json(coordinates_json, orient = 'index')

In [3]:
df = pd.concat([coordinates_df, id_timestamp_df], axis = 1, join_axes = [coordinates_df.index])
df

,x,y,z,tagId,timestamp
0,3323.0,-608.0,0.0,26383,2018-09-19 17:55:18.890000105
1,2675.0,-731.0,0.0,26457,2018-09-19 17:55:18.937999964
2,1990.0,-1172.0,0.0,26376,2018-09-19 17:55:19.006999969
3,3400.0,-1294.0,0.0,26395,2018-09-19 17:55:19.065999985
4,2044.0,-624.0,0.0,26382,2018-09-19 17:55:19.117000103
5,3304.0,-595.0,0.0,26383,2018-09-19 17:55:19.164999962
6,2710.0,-681.0,0.0,26457,2018-09-19 17:55:19.213999987
7,1964.0,-1106.0,0.0,26376,2018-09-19 17:55:19.250999927
8,3424.0,-1457.0,0.0,26395,2018-09-19 17:55:19.299000025
9,2045.0,-559.0,0.0,26382,2018-09-19 17:55:19.351000071


In [4]:
#Make separate dataframes of each tag
df1 = df[df['tagId']== 26395]
df2 = df[df['tagId']== 26376]
df3 = df[df['tagId']== 26382]
df4 = df[df['tagId']== 26457]
df5 = df[df['tagId']== 26383]

In [5]:
#Sort by timestamp
df1 = df1.sort_values(by='timestamp')
df2 = df2.sort_values(by='timestamp')
df3 = df3.sort_values(by='timestamp')
df4 = df4.sort_values(by='timestamp')
df5 = df5.sort_values(by='timestamp')

In [6]:
# Reset index
df1 = df1.reset_index(drop = True)
df2 = df2.reset_index(drop = True)
df3 = df3.reset_index(drop = True)
df4 = df4.reset_index(drop = True)
df5 = df5.reset_index(drop = True)
df1

,x,y,z,tagId,timestamp
0,3400.0,-1294.0,0.0,26395,2018-09-19 17:55:19.065999985
1,3424.0,-1457.0,0.0,26395,2018-09-19 17:55:19.299000025
2,3390.0,-1363.0,0.0,26395,2018-09-19 17:55:19.588999987
3,3381.0,-1367.0,0.0,26395,2018-09-19 17:55:19.821000099
4,3368.0,-1344.0,0.0,26395,2018-09-19 17:55:20.086999893
5,3366.0,-1431.0,0.0,26395,2018-09-19 17:55:20.323999882
6,3364.0,-1428.0,0.0,26395,2018-09-19 17:55:20.631999969
7,3344.0,-1344.0,0.0,26395,2018-09-19 17:55:20.865999937
8,3382.0,-1362.0,0.0,26395,2018-09-19 17:55:21.131000042
9,3358.0,-1222.0,0.0,26395,2018-09-19 17:55:21.365000010


In [7]:
#Find max and min
max_x_1 = df1['x'].max()
max_y_1 = df1['y'].max()
min_x_1 = df1['x'].min()
min_y_1 = df1['y'].min()

#print(max_x_1, max_y_1, min_x_1, min_y_1)

max_x_2 = df2['x'].max()
max_y_2 = df2['y'].max()
min_x_2 = df2['x'].min()
min_y_2 = df2['y'].min()

print(max_x_2, max_y_2, min_x_2, min_y_2)

print(df1.at[0,'x'])

2057.0 -1062.0 1901.0 -1252.0
3400.0


In [8]:
#Distance formula

import math

def distance(x1, y1, x2, y2) :
    dist_x = x1 - x2
    dist_y = y1 - y2
    distance = math.sqrt(dist_x**2 + dist_y**2)
    return distance

dist = []
for index, row in df1.iterrows():
    disttemp = distance(df1.loc[index,'x'],df1.loc[index,'y'],df2.loc[index,'x'],df2.loc[index,'y'])
    dist.append(disttemp)

df1['dist_to_2'] = dist
df1

,x,y,z,tagId,timestamp,dist_to_2
0,3400.0,-1294.0,0.0,26395,2018-09-19 17:55:19.065999985,1415.268172
1,3424.0,-1457.0,0.0,26395,2018-09-19 17:55:19.299000025,1501.599481
2,3390.0,-1363.0,0.0,26395,2018-09-19 17:55:19.588999987,1450.868016
3,3381.0,-1367.0,0.0,26395,2018-09-19 17:55:19.821000099,1443.698722
4,3368.0,-1344.0,0.0,26395,2018-09-19 17:55:20.086999893,1397.008948
5,3366.0,-1431.0,0.0,26395,2018-09-19 17:55:20.323999882,1406.080012
6,3364.0,-1428.0,0.0,26395,2018-09-19 17:55:20.631999969,1411.592363
7,3344.0,-1344.0,0.0,26395,2018-09-19 17:55:20.865999937,1409.193031
8,3382.0,-1362.0,0.0,26395,2018-09-19 17:55:21.131000042,1421.172052
9,3358.0,-1222.0,0.0,26395,2018-09-19 17:55:21.365000010,1418.185460


In [9]:
df2.to_csv('test_2.csv')